### 检查cuda版本型号，貌似10.2的还没适配cupy

In [0]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.0.130


In [0]:
!nvidia-smi

Sat Mar  7 02:59:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 检查cuda是否有用

In [0]:
import cupy
cupy.cuda.device.get_device_id()

0

#### 没用的话，需要安装10.0cuda或者重新分配gpu

In [0]:
'''
!apt-get update && sudo apt-get install -y build-essential git libgfortran3
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
!sudo dpkg -i cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-<version>/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda
'''

### 安装一些colab没有的库

In [0]:
!pip install ipdb
!pip install visdom
!pip3 install git+https://github.com/pytorch/tnt.git@master


### 为了在colab上使用visdom

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('./ngrok http 8097 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


import visdom
vis = visdom.Visdom(port='8097', env='test')

--2020-03-07 03:01:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.114.213, 34.226.171.201, 34.192.215.160, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.114.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  18.2MB/s    in 0.7s    

2020-03-07 03:01:55 (18.2 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
http://063e6814.ngrok.io


Setting up a new session...


### 导入必要的库

In [0]:
import os
import ipdb
import torch
import torch.nn as nn

import sys 
sys.path.append("/content/drive/My Drive/img_attack_with_attention") 

from utils.config import opt
from data.util import read_image
from data.dataset import Dataset
from data.dataset import inverse_normalize
from torch.utils import data as data_
from model import FasterRCNNVGG16
from functools import partial
from trainer import BRFasterRcnnTrainer
from utils import array_tool as at
from utils.vis_tool import visdom_bbox
from torch.autograd import Variable
from tqdm import tqdm
from data.dataset import preprocess, pytorch_normalze
import attacks

layer_idies = [15, 20]
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/content/drive/My Drive/img_attack_with_attention/model/utils/nms/non_maximum_suppression.py:15: UserWarning: 
    the python code for non_maximum_suppression is about 2x slow
    It is strongly recommended to build cython code:
    `cd model/utils/nms/; python3 build.py build_ext --inplace
  `cd model/utils/nms/; python3 build.py build_ext --inplace''')


### 下载PASCAL VOC 2007数据集并解压

In [0]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar

In [0]:
!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar
!tar xvf VOCdevkit_08-Jun-2007.tar

### 可重新定义相关路径

In [0]:
data_dir = '/content/VOCdevkit/VOC2007'
attacker_path = '/content/drive/My Drive/img_attack_with_attention/checkpoints/10.path'
save_path_HOME = '/content/drive/My Drive/img_attack_with_attention/sdd'

In [0]:
def store(model):
    """
    make hook for feature map
    """

    def hook(module, input, output, key):
        for layer_idx in layer_idies:
            if key == layer_idx:
                model.feature_maps[key] = output[0]

    for idx, layer in enumerate(model._modules.get('extractor')):
        layer.register_forward_hook(partial(hook, key=idx))


In [0]:
def train(**kwargs):
    opt._parse(kwargs)
    # opt.caffe_pretrain = True
    opt.voc_data_dir = '/content/VOCdevkit/VOC2007'
    opt.load_path = '/content/drive/My Drive/model_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth'
    TrainResume = True
    dataset = Dataset(opt)
    print('load dataset')
    dataloader = data_.DataLoader(dataset,
                                  batch_size=1,
                                  shuffle=True,
                                  pin_memory=True,
                                  num_workers=opt.num_workers)
    target_img_path = '/content/drive/My Drive/img_attack_with_attention/target_img.jpg'
    target_img = read_image(target_img_path) / 255
    target_img = torch.from_numpy(pytorch_normalze(target_img))
    target_img = torch.unsqueeze(target_img, 0).numpy()

    attacker = attacks.Blade_runner(train_BR=True)

    if TrainResume:
        #attacker.load('/content/drive/My Drive/img_attack_with_attention/checkpoints/attack_03041223_2500.path')
        attacker.load('/content/drive/My Drive/img_attack_with_attention/checkpoints/10.path')
    # attacker = attacks_no_target.Blade_runner(train_BR=True)
    faster_rcnn = FasterRCNNVGG16().eval()
    faster_rcnn.cuda()
    store(faster_rcnn)
    trainer = BRFasterRcnnTrainer(faster_rcnn, attacker, layer_idx=layer_idies, attack_mode=True).cuda()

    if opt.load_path:
        trainer.load(opt.load_path)
        print('from %s Load model parameters' % opt.load_path)

    trainer.vis.text(dataset.db.label_names, win='labels')
    target_features_list = list()

    img_feature = trainer.faster_rcnn(torch.from_numpy(target_img).cuda())
    del img_feature
    target_features = trainer.faster_rcnn.feature_maps
    for target_feature_idx in target_features:
        target_features_list.append(target_features[target_feature_idx].cpu().detach().numpy())
    del target_features

    for epoch in range(opt.epoch):
        trainer.reset_meters(BR=True)
        for ii, (img, bbox_, label_, scale) in tqdm(enumerate(dataloader)):
            scale = at.scalar(scale)
            img, bbox, label = img.cuda().float(), bbox_.cuda(), label_.cuda()
            img, bbox, label = Variable(img), Variable(bbox), Variable(label)
            rois, roi_scores = faster_rcnn(img, flag=True)
            if len(rois) != len(roi_scores):
                print('The generated ROI and ROI score lengths are inconsistent')
            trainer.train_step(img, bbox, label, scale, target_features_list, rois=rois, roi_scores=roi_scores)

            if (ii) % opt.plot_every == 0:
                if os.path.exists(opt.debug_file):
                    ipdb.set_trace()

                # plot loss
                trainer.vis.plot_many(trainer.get_meter_data(BR=True))

                # plot groud truth bboxes
                ori_img_ = inverse_normalize(at.tonumpy(img[0]))
                print('ori_img_',ori_img_)
                gt_img = visdom_bbox(ori_img_,
                            at.tonumpy(bbox_[0]),
                            at.tonumpy(label_[0]))
                trainer.vis.img('gt_img', gt_img)

                # plot predicted bboxes
                _bboxes, _labels, _scores = trainer.faster_rcnn.predict([ori_img_], visualize=True)
                print(_bboxes)
                pred_img = visdom_bbox(ori_img_,
                                       at.tonumpy(_bboxes[0]),
                                       at.tonumpy(_labels[0]).reshape(-1),
                                       at.tonumpy(_scores[0]))
                trainer.vis.img('pred_img', pred_img)
                if trainer.attacker is not None:
                    adv_img = trainer.attacker.perturb(img, rois=rois, roi_scores=roi_scores)
                    adv_img_ = inverse_normalize(at.tonumpy(adv_img[0]))
                    _bboxes, _labels, _scores = trainer.faster_rcnn.predict([adv_img_], visualize=True)
                    adv_pred_img = visdom_bbox(adv_img_,
                                               at.tonumpy(_bboxes[0]),
                                               at.tonumpy(_labels[0]).reshape(-1),
                                               at.tonumpy(_scores[0]))
                    trainer.vis.img('adv_img', adv_pred_img)
                # rpn confusion matrix(meter)
                trainer.vis.text(str(trainer.rpn_cm.value().tolist()), win='rpn_cm')
                # roi confusion matrix
                trainer.vis.img('roi_cm', at.totensor(trainer.roi_cm.conf, False).float())

            if ii % 500 == 0 and ii != 0:
                best_path = trainer.save(epochs=ii, save_rcnn=False)
                print('best path is %s' % best_path)
        if epoch % 2 == 0:
            best_path = trainer.save(epochs=epoch, save_rcnn=False)



In [0]:
import warnings


if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    train()